Mounting a Google Drive to Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Selecting the latest version of Tensorflow

In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


Importing Tensorflow and checking what version we're using, as well as checking for a GPU

In [3]:
import tensorflow as tf
print(tf.test.gpu_device_name())
print(tf.__version__)

/device:GPU:0
2.1.0


Import any other necessary libraries

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.constraints import MaxNorm

Create a Sequential object

In [0]:
classifier = Sequential([
    Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu', padding = 'same'),
    Dropout(0.2),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
    MaxPooling2D(pool_size = (2, 2)),
    Dropout(0.3),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation = 'relu', padding = 'same'),
    Dropout(0.3),
    BatchNormalization(),
    Flatten(),
    Dropout(0.3),
    Dense(units = 128, activation = 'relu', kernel_constraint = MaxNorm(3)),
    Dropout(0.3),
    BatchNormalization(),
    Dense(units = 6, activation = 'softmax')
])

Checkpoint callback

In [0]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('./', save_best_only=True)

Compile the Sequential object

In [0]:
classifier.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy', 
    metrics = ['accuracy']
)

Create the training data generator

In [0]:
train_datagen = ImageDataGenerator(
    rescale = 1./255, 
    shear_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True
)

Create the testing data generator

In [0]:
test_datagen = ImageDataGenerator(rescale = 1./255)

Setup the training dataset

In [10]:
training_set = train_datagen.flow_from_directory(
    "/content/drive/My Drive/Colab Notebooks/FRFD/train", 
    target_size = (64, 64), 
    batch_size = 128, 
    class_mode = 'categorical'
)

Found 10955 images belonging to 6 classes.


Setup the testing dataset

In [11]:
test_set = test_datagen.flow_from_directory(
    "/content/drive/My Drive/Colab Notebooks/FRFD/test", 
    target_size = (64, 64), 
    batch_size = 128, 
    class_mode = 'categorical'
)

Found 2698 images belonging to 6 classes.


Showing the model summary

In [12]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        2

Setup the fit generator for the Sequential object & apply it

In [0]:
classifier.fit(
  training_set,
  epochs = 30,
  callbacks = [checkpoint],
  validation_data = test_set
  #steps_per_epoch = 100,
  #validation_steps = 100
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 86 steps, validate for 22 steps
Epoch 1/30
17/86 [====>.........................] - ETA: 29:09 - loss: 1.0866 - accuracy: 0.6443

Test/evaluate the final model

In [0]:
#classes = training_set.class_indices
#print("Classes used: %s\n" % classes)
#test_img = image.load_img("/content/orange.png")
#test_img = image.img_to_array(test_img)
#test_img = np.array(test_img, axis = 0)
#result = classifier.predict(test_img)
#print("Prediction result: %s\n" % result)
scores = classifier.evaluate(test_set, verbose = 0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

Save the model

In [0]:
import base64

def save_model_signed(model, path):
    @tensorflow.function(input_signature = [tensorflow.TensorSpec(shape = [None, ], dtype = tensorflow.string)])
    def preprocess_and_evaluate(b64_img):
        img = tensorflow.image.decode_image(b64_img[0], dtype = tf.uint8)
        img.set_shape((None, None, 3))
        img = tensorflow.image.resize(img, [64, 64])
        img = tensorflow.reshape(img, (-1, 64, 64, 3))
        img = tensorflow.cast(img, dtype = tf.float32) / 255

        return model(img)
    
    tensorflow.saved_model.save(model, path, signatures = preprocess_and_evaluate)

save_model_signed(classifier, "/content/drive/My Drive/Colab Notebooks/")